In [ ]:
#Calculate cost of all channels
import json
import os
import pinecone
import openai
#INSERT API KEYS HERE
pinecone.init(api_key="", environment="us-west4-gcp")
index = pinecone.Index(index_name='gpt-dashboard-db')
os.environ['OPENAI_API_KEY'] = ""
openai.api_key = ""
embed_id = 1

DIRECTORY = "./chunks"

total_cost = 0

for filename in os.listdir(DIRECTORY):
    if filename.endswith(".json"):
        with open(os.path.join(DIRECTORY, filename), 'r') as f:
            chunks = json.load(f)

        total_input, obj_num = 0, 0
        for obj in chunks:
            total_input += obj["token_count"]
            obj_num += 1

        #approx cost of sys prompt
        sys_prompt_cost = obj_num*200
        qa_input_cost = ((total_input+sys_prompt_cost)/1000)*0.0015
        qa_output_cost = ((obj_num*900)/1000)*0.002
        total_qa_cost = qa_input_cost+ qa_output_cost
        embedding_cost = ((obj_num*900)/1000)*0.0001
        file_cost = total_qa_cost+embedding_cost
        total_cost += file_cost

print(f"Grand Total: $ {round(total_cost, 3)}")


In [ ]:
#Get filenames and names of channels  
import os
import json

DIRECTORY = "./chunks"

# Lists to store filenames and modified filenames
filenames = []
modified_filenames = []

# Iterate over all files in the directory
for filename in os.listdir(DIRECTORY):
    if filename.endswith(".json"):
        # Add the filename to the list
        filenames.append(filename)
        
        # Remove the .json extension and prepend a '#' to the filename
        modified_name = '#' + filename[:-5]
        modified_filenames.append(modified_name)

# Save the lists to JSON files
with open('filenames.json', 'w') as f:
    json.dump(filenames, f)

with open('modified_filenames.json', 'w') as f:
    json.dump(modified_filenames, f)


In [ ]:
#DELETE NAMESPACE (need to import pinecone index)
index.delete(delete_all=True, namespace="")

In [ ]:
#retrieve embeddings by id 

ALL_CHANNELS = [ ] #<---- insert channel names e.g "#offer-letter"


def pinecone_query(query):
    response = openai.Embedding.create(
        input=query,
        model="text-embedding-ada-002",
    )
    embeddings1 = response["data"][0]["embedding"]

    answer = index.query(
        vector=embeddings1,
        include_metadata=True,
        top_k=1,
        namespace='slack'
    )

    return answer

for c in ALL_CHANNELS:
    for i in range(1,50, 10):
        answer = index.query(
            id=f"{c}_{i}",
            include_metadata=True,
            top_k=1,
            namespace='slack'
        )
        print("CHANNEL", c)
        print(answer['matches'])
        print('-----------------------------')
